<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Yvonne", "transactions": [{"transaction-id": 1899, "amount": 94}, {"transaction-id": 2661, "amount": -72}, {"transaction-id": 3757, "amount": -54}, {"transaction-id": 4224, "amount": -20}, {"transaction-id": 4917, "amount": -19}, {"transaction-id": 5203, "amount": 28}, {"transaction-id": 5266, "amount": 0}, {"transaction-id": 7267, "amount": -87}, {"transaction-id": 7295, "amount": -36}, {"transaction-id": 7616, "amount": 24}, {"transaction-id": 7846, "amount": 14}, {"transaction-id": 8430, "amount": -25}, {"transaction-id": 9511, "amount": 28}, {"transaction-id": 9786, "amount": 29}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Yvonne", "transactions": [{"transaction-id": 1899, "amount": 94}, {"transaction-id": 2661, "amount": -72}, {"transaction-id": 3757, "amount": -54}, {"transaction-id": 4224, "amount": -20}, {"transaction-id": 4917, "amount": -19}, {"transaction-id": 5203, "amount": 28}, {"transaction-id": 5266, "amount": 0}, {"transaction-id": 7267, "amount": -87}, {"transaction-id": 7295, "amount": -36}, {"transaction-id": 7616, "amount": 24}, {"transaction-id": 7846, "amount": 14}, {"transaction-id": 8430, "amount": -25}, {"transaction-id": 9511, "amount": 28}, {"transaction-id": 9786, "amount": 29}]}\n',
 '{"id": 1, "name": "Yvonne", "transactions": [{"transaction-id": 2739, "amount": 221}, {"transaction-id": 3374, "amount": 283}, {"transaction-id": 4307, "amount": 115}, {"transaction-id": 6428, "amount": 285}, {"transaction-id": 6884, "amount": 279}, {"transaction-id": 6986, "amount": 158}, {"transaction-id": 8460, "amount": 320}, {"transaction-id": 9123, "amount": 193}]}\n',
 '

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Yvonne',
  'transactions': [{'transaction-id': 1899, 'amount': 94},
   {'transaction-id': 2661, 'amount': -72},
   {'transaction-id': 3757, 'amount': -54},
   {'transaction-id': 4224, 'amount': -20},
   {'transaction-id': 4917, 'amount': -19},
   {'transaction-id': 5203, 'amount': 28},
   {'transaction-id': 5266, 'amount': 0},
   {'transaction-id': 7267, 'amount': -87},
   {'transaction-id': 7295, 'amount': -36},
   {'transaction-id': 7616, 'amount': 24},
   {'transaction-id': 7846, 'amount': 14},
   {'transaction-id': 8430, 'amount': -25},
   {'transaction-id': 9511, 'amount': 28},
   {'transaction-id': 9786, 'amount': 29}]},
 {'id': 1,
  'name': 'Yvonne',
  'transactions': [{'transaction-id': 2739, 'amount': 221},
   {'transaction-id': 3374, 'amount': 283},
   {'transaction-id': 4307, 'amount': 115},
   {'transaction-id': 6428, 'amount': 285},
   {'transaction-id': 6884, 'amount': 279},
   {'transaction-id': 6986, 'amount': 158},
   {'transaction-id': 8460, 'amo

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 3,
  'name': 'Alice',
  'transactions': [{'transaction-id': 10, 'amount': 3812},
   {'transaction-id': 126, 'amount': 3102},
   {'transaction-id': 210, 'amount': 3579},
   {'transaction-id': 258, 'amount': 3127},
   {'transaction-id': 269, 'amount': 3878},
   {'transaction-id': 315, 'amount': 3516},
   {'transaction-id': 405, 'amount': 3398},
   {'transaction-id': 428, 'amount': 3797},
   {'transaction-id': 487, 'amount': 4016},
   {'transaction-id': 536, 'amount': 4067},
   {'transaction-id': 541, 'amount': 3869},
   {'transaction-id': 627, 'amount': 3924},
   {'transaction-id': 638, 'amount': 3832},
   {'transaction-id': 785, 'amount': 3442},
   {'transaction-id': 837, 'amount': 3739},
   {'transaction-id': 860, 'amount': 3610},
   {'transaction-id': 1148, 'amount': 3685},
   {'transaction-id': 1239, 'amount': 3184},
   {'transaction-id': 1240, 'amount': 4322},
   {'transaction-id': 1249, 'amount': 3584},
   {'transaction-id': 1258, 'amount': 3596},
   {'transaction-id': 1442

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 187},
 {'name': 'Alice', 'count': 6},
 {'name': 'Alice', 'count': 34},
 {'name': 'Alice', 'count': 85},
 {'name': 'Alice', 'count': 19})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(187, 6, 34, 85, 19)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

48.17550274223035

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 10, 'amount': 3812},
  {'transaction-id': 126, 'amount': 3102},
  {'transaction-id': 210, 'amount': 3579},
  {'transaction-id': 258, 'amount': 3127},
  {'transaction-id': 269, 'amount': 3878},
  {'transaction-id': 315, 'amount': 3516},
  {'transaction-id': 405, 'amount': 3398},
  {'transaction-id': 428, 'amount': 3797},
  {'transaction-id': 487, 'amount': 4016},
  {'transaction-id': 536, 'amount': 4067},
  {'transaction-id': 541, 'amount': 3869},
  {'transaction-id': 627, 'amount': 3924},
  {'transaction-id': 638, 'amount': 3832},
  {'transaction-id': 785, 'amount': 3442},
  {'transaction-id': 837, 'amount': 3739},
  {'transaction-id': 860, 'amount': 3610},
  {'transaction-id': 1148, 'amount': 3685},
  {'transaction-id': 1239, 'amount': 3184},
  {'transaction-id': 1240, 'amount': 4322},
  {'transaction-id': 1249, 'amount': 3584},
  {'transaction-id': 1258, 'amount': 3596},
  {'transaction-id': 1442, 'amount': 3527},
  {'transaction-id': 1513, 'amount': 4385},
  {'t

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 10, 'amount': 3812},
 {'transaction-id': 126, 'amount': 3102},
 {'transaction-id': 210, 'amount': 3579})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(3812, 3102, 3579)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1815.8027094717668

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 131), ('Alice', 131), ('Alice', 142), ('Alice', 143), ('Bob', 97), ('Bob', 99), ('Bob', 106), ('Bob', 109), ('Charlie', 86), ('Charlie', 87), ('Charlie', 91), ('Charlie', 92), ('Dan', 120), ('Dan', 120), ('Dan', 129), ('Dan', 130), ('Edith', 119), ('Edith', 120), ('Edith', 130), ('Edith', 130), ('Frank', 147), ('Frank', 159), ('Frank', 307), ('George', 136), ('George', 139), ('George', 145), ('George', 150), ('Hannah', 119), ('Hannah', 119), ('Hannah', 130), ('Hannah', 130), ('Ingrid', 67), ('Ingrid', 67), ('Ingrid', 71), ('Ingrid', 73), ('Jerry', 95), ('Jerry', 96), ('Jerry', 102), ('Jerry', 103), ('Kevin', 144), ('Kevin', 144), ('Kevin', 156), ('Kevin', 156), ('Laura', 181), ('Laura', 185), ('Laura', 197), ('Laura', 197), ('Michael', 48), ('Michael', 48), ('Michael', 52), ('Michael', 52), ('Norbert', 108), ('Norbert', 109), ('Norbert', 214), ('Oliver', 106), ('Oliver', 106), ('Oliver', 115), ('Oliver', 117), ('Patricia', 96), ('Patricia', 96), ('Patricia', 104), ('Patricia

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 547), ('Bob', 411), ('Charlie', 356), ('Dan', 499), ('Edith', 499), ('Frank', 613), ('George', 570), ('Hannah', 498), ('Ingrid', 278), ('Jerry', 396), ('Kevin', 600), ('Laura', 760), ('Michael', 200), ('Norbert', 431), ('Oliver', 444), ('Patricia', 400), ('Quinn', 294), ('Ray', 617), ('Sarah', 395), ('Tim', 300), ('Ursula', 500), ('Victor', 548), ('Wendy', 445), ('Xavier', 499), ('Yvonne', 460), ('Zelda', 549)]
CPU times: user 127 ms, sys: 7.67 ms, total: 135 ms
Wall time: 467 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Yvonne,"[{'transaction-id': 1899, 'amount': 94}, {'tra..."
1,1,Yvonne,"[{'transaction-id': 2739, 'amount': 221}, {'tr..."
2,2,Edith,"[{'transaction-id': 4950, 'amount': 406}, {'tr..."
3,3,Alice,"[{'transaction-id': 10, 'amount': 3812}, {'tra..."
4,4,Dan,"[{'transaction-id': 4517, 'amount': 1208}, {'t..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 149 ms, sys: 20.9 ms, total: 170 ms
Wall time: 1.34 s


name
Alice      547
Bob        411
Charlie    356
Dan        499
Edith      499
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Yvonne', 'amount': 94, 'transaction-id': 1899},
 {'id': 0, 'name': 'Yvonne', 'amount': -72, 'transaction-id': 2661},
 {'id': 0, 'name': 'Yvonne', 'amount': -54, 'transaction-id': 3757})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Yvonne,94,1899
1,0,Yvonne,-72,2661
2,0,Yvonne,-54,3757
3,0,Yvonne,-20,4224
4,0,Yvonne,-19,4917


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 169 ms, sys: 9.09 ms, total: 178 ms
Wall time: 1.08 s


name
Alice       26352
Bob         16699
Charlie     14372
Dan         35471
Edith       21745
Frank       17231
George      28264
Hannah      23102
Ingrid      13255
Jerry       24249
Kevin       22948
Laura       40803
Michael      8067
Norbert     18657
Oliver      13279
Patricia     7601
Quinn       12679
Ray         28154
Sarah       10978
Tim         13092
Ursula      19027
Victor      29054
Wendy        8087
Xavier      14070
Yvonne      13189
Zelda       19575
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()